In [1]:
!pip install tensorflowjs #model 파일 변환을 위한 tenserflowjs 설치

     |████████████████████████████████| 77 kB 3.2 MB/s 


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import feature_column
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers
from sklearn.preprocessing import MultiLabelBinarizer

import matplotlib.pyplot as plt
from google.colab import files  #csv 파일 업로드를 위함

In [3]:
seed = 10
np.random.seed(seed)

csvFile = files.upload() #데이터 파일 업로드


Saving steamEngine_dataset.csv to steamEngine_dataset.csv


In [4]:
import io
data = pd.read_csv(io.BytesIO(csvFile['steamEngine_dataset.csv']))
print(data.shape)

(3625, 55)


In [21]:
label = data['label']
features = data.drop(['label'],axis=1)

features

,0_position_x,0_position_y,0_score,1_position_x,1_position_y,1_score,2_position_x,2_position_y,2_score,3_position_x,3_position_y,3_score,4_position_x,4_position_y,4_score,5_position_x,5_position_y,5_score,6_position_x,6_position_y,6_score,7_position_x,7_position_y,7_score,8_position_x,8_position_y,8_score,9_position_x,9_position_y,9_score,10_position_x,10_position_y,10_score,11_position_x,11_position_y,11_score,12_position_x,12_position_y,12_score,13_position_x,13_position_y,13_score,14_position_x,14_position_y,14_score,15_position_x,15_position_y,15_score,16_position_x,16_position_y,16_score,img_height,img_width,pose_score
0,81.903974,30.619032,0.997260,88.698304,25.796371,0.995816,80.330977,27.292648,0.991151,95.669012,29.374220,0.877343,77.334821,32.697586,0.752247,108.307397,51.567008,0.982155,62.979313,52.868835,0.983104,131.688178,30.968672,0.896813,37.813347,31.567016,0.952245,168.364938,121.535109,0.327864,30.727566,25.412799,0.473601,101.351764,129.291185,0.990708,66.760484,129.296624,0.982278,82.918226,195.597637,0.898004,97.616050,198.643114,0.782822,180.973931,141.703851,0.435073,73.597452,228.755924,0.528034,224,224,0.814501
1,115.630282,59.342441,0.941545,118.484020,55.611572,0.935903,111.902980,54.752099,0.869579,122.941344,56.951545,0.583492,107.475520,57.154329,0.720850,131.668256,71.431969,0.963720,97.811513,73.349869,0.974255,157.667866,69.346476,0.910040,76.080640,69.422370,0.953497,135.322743,55.921566,0.592300,92.827569,58.725950,0.678341,106.135338,122.548902,0.976440,127.205466,125.892992,0.989493,97.635155,167.548988,0.950024,127.293202,167.800228,0.971158,90.877179,203.770187,0.828057,133.987939,202.901318,0.879179,224,224,0.865757
2,115.299949,59.633766,0.911103,118.276242,55.894109,0.914744,112.086247,55.124096,0.838578,121.761267,57.263099,0.557099,108.081046,56.437779,0.624158,131.148565,71.711212,0.932144,97.144703,72.759651,0.960012,157.405853,68.641903,0.881088,75.580565,68.080432,0.946032,134.057270,54.779765,0.524108,94.815896,54.727728,0.621856,107.171926,122.672986,0.974222,127.139567,126.011318,0.985689,97.136989,169.056099,0.951171,127.785430,168.478236,0.962705,92.469434,208.177425,0.816080,133.931895,203.886730,0.882941,224,224,0.840219
3,115.835958,57.624087,0.872733,118.721349,55.408804,0.847805,112.905509,54.023171,0.782052,120.709995,57.661871,0.611752,118.147769,57.218167,0.581848,104.552239,75.504206,0.883456,99.187585,73.502181,0.891440,156.416504,70.920504,0.743468,75.745459,68.028577,0.947282,135.212623,55.536593,0.511806,93.856265,54.682888,0.574665,101.564223,123.172582,0.978458,128.139609,125.957721,0.991322,97.340279,167.527150,0.944216,127.469341,167.117473,0.973067,92.710800,207.481981,0.787816,133.261668,204.689527,0.891254,224,224,0.812614
4,115.362470,58.495170,0.910149,118.041320,55.992539,0.906762,112.435252,54.455224,0.855118,120.729146,57.164251,0.619286,117.189168,57.716997,0.636112,130.923059,71.122996,0.901127,97.385145,72.312356,0.951519,158.124158,68.682553,0.827213,76.531206,69.122114,0.944457,135.274638,53.888769,0.453957,93.101506,55.351573,0.619217,104.880830,122.260370,0.972719,127.092939,125.713303,0.990550,97.453777,166.012391,0.960905,126.661530,166.986286,0.973921,93.557505,205.258253,0.807394,133.854306,203.843826,0.884938,224,224,0.836197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3620,139.216195,25.940933,0.733553,142.559074,12.285407,0.675106,134.343342,6.902498,0.544589,156.392246,7.552025,0.932795,130.215539,12.283008,0.483606,190.567558,44.636654,0.960986,119.360842,55.409852,0.925901,218.312447,86.603874,0.982274,120.804129,103.361935,0.906746,199.999595,115.058430,0.971579,127.626860,115.075587,0.903934,186.467937,137.956547,0.555415,146.606734,131.708779,0.824549,213.294776,136.392274,0.947548,135.518937,152.985502,0.876941,190.701537,219.240352,0.798295,194.150799,22

In [22]:
def build_model():
  model = keras.Sequential([
                            keras.Input(shape = [len(features.keys())]),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(3, activation = 'softmax')
  ])

  model.compile(
      loss='categorical_crossentropy',
      optimizer = 'adam',
      metrics=['acc'])
  return model

In [23]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                3520      
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 3)                 195       
                                                                 
Total params: 12,035
Trainable params: 12,035
Non-trainable params: 0
_________________________________________________________________


In [27]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(label.str.split(","))
mlb.classes_

array(['nothing', 'steamHi', 'steamready'], dtype=object)

In [28]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.2)

In [29]:
from sklearn.utils import validation
model.fit( train_x, train_y, epochs = 30 )

Epoch 1/30
91/91 [==============================] - 1s 2ms/step - loss: 2.5058 - acc: 0.7848
Epoch 2/30
91/91 [==============================] - 0s 2ms/step - loss: 0.3494 - acc: 0.9128
Epoch 3/30
91/91 [==============================] - 0s 2ms/step - loss: 0.2246 - acc: 0.9272
Epoch 4/30
91/91 [==============================] - 0s 2ms/step - loss: 0.1997 - acc: 0.9414
Epoch 5/30
91/91 [==============================] - 0s 2ms/step - loss: 0.1850 - acc: 0.9424
Epoch 6/30
91/91 [==============================] - 0s 2ms/step - loss: 0.1531 - acc: 0.9548
Epoch 7/30
91/91 [==============================] - 0s 2ms/step - loss: 0.1449 - acc: 0.9552
Epoch 8/30
91/91 [==============================] - 0s 2ms/step - loss: 0.1432 - acc: 0.9555
Epoch 9/30
91/91 [==============================] - 0s 2ms/step - loss: 0.0678 - acc: 0.9734
Epoch 10/30
91/91 [==============================] - 0s 2ms/step - loss: 0.0618 - acc: 0.9803
Epoch 11/30
91/91 [==============================] - 0s 2ms/step - lo

In [30]:
model.evaluate(test_x, test_y) #테스트 데이터로 검증
mlb.classes_

23/23 [==============================] - 0s 1ms/step - loss: 0.0508 - acc: 0.9793


array(['nothing', 'steamHi', 'steamready'], dtype=object)

In [ ]:
# 테스트를 위한 코드
# csvTestFile =  files.upload() 
# test = pd.read_csv(io.BytesIO(csvTestFile['csv_downloadfile (9).csv']))
# test
# proba = model.predict(test)
# idx = np.argmax(proba)
# proba

Save Model


In [31]:
import tensorflowjs as tfjs #저장된 모델을 불러오기 위함
tfjs.converters.save_keras_model(model,artifacts_dir="./push_up_model1.json") #tensorflow.js 사용 가능 형태로 저장
model.save('push_up_model.h5') #케라스 모델 저장

'/content'